# Ring spot helper demonstration


This notebook demonstrates the analytic ring spot helper using the same mesh API that powers ``add_spot`` and ``add_pulsation``.
We'll build a base ``IcosphereModel``, perturb its temperature and gravity columns with ``add_ring_spot``, and forward the spotted
mesh through the ``PhysicalLineEmulator`` to see how the bright ring reshapes a Ca II IRT-like spectral line.


## Imports and configuration


In [1]:
import jax
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

from spice.models import IcosphereModel
from spice.models.spots import add_ring_spot, RingSpotConfig
from spice.utils.ring_spot import ring_spot_weights, ca_irt_scale_map
from spice.spectrum import PhysicalLineEmulator


## Build a base mesh and apply ``add_ring_spot``


In [2]:
class CaIrtEmulator(PhysicalLineEmulator):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def intensity(self, wavelengths, mu, params):
        return super().intensity(wavelengths, mu, params[:-1])*params[-1]

In [3]:
cfg = RingSpotConfig(
    sigma_umb_deg=10.0,
    theta0_deg=30.0,
    sigma_plage_deg=4.0,
    umbra_delta=-800.0,
    plage_delta=100.0
    )
base_temp = 5700.0
base_logg = 4.35

mesh = IcosphereModel.construct(
    2000,
    1.0,
    1.0,
    jnp.array([base_temp, base_logg, 1.0]),
    ['teff', 'logg', 'ca_irt_scale']
)

spot_center_theta = jnp.deg2rad(35.0)
spot_center_phi = jnp.deg2rad(55.0)

teff_mesh = add_ring_spot(
    mesh,
    spot_center_theta=spot_center_theta,
    spot_center_phi=spot_center_phi,
    param_index=0,
    config=cfg,
)

spotted_mesh = add_ring_spot(
    teff_mesh,
    spot_center_theta=spot_center_theta,
    spot_center_phi=spot_center_phi,
    param_index=1,
    config=cfg,
    umbra_delta=-0.2,
    plage_delta=0.05,
)

normals = mesh.d_centers / jnp.linalg.norm(mesh.d_centers, axis=1, keepdims=True)
spot_axis = jnp.array([
    jnp.sin(spot_center_theta) * jnp.cos(spot_center_phi),
    jnp.sin(spot_center_theta) * jnp.sin(spot_center_phi),
    jnp.cos(spot_center_theta),
])
w_umb, w_plage = ring_spot_weights(normals, spot_axis, cfg)

normals_np = np.array(normals)
lon = np.degrees(np.arctan2(normals_np[:, 1], normals_np[:, 0]))
lat = np.degrees(np.arcsin(normals_np[:, 2]))

temp_delta = np.array(spotted_mesh.parameters[:, 0] - mesh.parameters[:, 0])
logg_delta = np.array(spotted_mesh.parameters[:, 1] - mesh.parameters[:, 1])


### Umbra and plage weights


In [4]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4), constrained_layout=True)
sc0 = axes[0].scatter(lon, lat, c=np.array(w_umb), s=10, cmap='magma', vmin=0.0)
axes[0].set_title('Umbra weight')
axes[0].set_xlabel('Longitude [deg]')
axes[0].set_ylabel('Latitude [deg]')
fig.colorbar(sc0, ax=axes[0], label='w_umb')

sc1 = axes[1].scatter(lon, lat, c=np.array(w_plage), s=10, cmap='viridis', vmin=0.0)
axes[1].set_title('Plage weight')
axes[1].set_xlabel('Longitude [deg]')
axes[1].set_ylabel('Latitude [deg]')
fig.colorbar(sc1, ax=axes[1], label='w_plage')
plt.show()


/var/folders/7r/n_x0ntj511v_0gt816mgrc1c0000gq/T/ipykernel_91368/2928994273.py:13: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


### Parameter perturbations


In [5]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4), constrained_layout=True)
sc0 = axes[0].scatter(lon, lat, c=temp_delta, s=10, cmap='coolwarm')
axes[0].set_title('Temperature perturbation [K]')
axes[0].set_xlabel('Longitude [deg]')
axes[0].set_ylabel('Latitude [deg]')
fig.colorbar(sc0, ax=axes[0], label='ΔT')

sc1 = axes[1].scatter(lon, lat, c=logg_delta, s=10, cmap='Spectral')
axes[1].set_title('log g perturbation [dex]')
axes[1].set_xlabel('Longitude [deg]')
axes[1].set_ylabel('Latitude [deg]')
fig.colorbar(sc1, ax=axes[1], label='Δlog g')
plt.show()


/var/folders/7r/n_x0ntj511v_0gt816mgrc1c0000gq/T/ipykernel_91368/2372763797.py:13: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


### Feature-specific scaling map (optional)
Use ``ca_irt_scale_map`` to create a per-element multiplier that you can
apply to any spectral mask after running your favorite emulator.


In [6]:
scale_map = ca_irt_scale_map(normals, spot_axis, cfg, plage_scale=0.6, umbra_scale=-0.4)

spotted_mesh = spotted_mesh._replace(parameters=jnp.concatenate([spotted_mesh.parameters[:,:2], scale_map[:, None]], axis=1))

fig, axes = plt.subplots(1, 2, figsize=(12, 4), constrained_layout=True)
axes[0].hist(np.array(scale_map), bins=30, color='tab:blue')
axes[0].set_xlabel('Scale factor')
axes[0].set_ylabel('Count')
axes[0].set_title('Distribution of mask scaling')

sc = axes[1].scatter(lon, lat, c=np.array(scale_map), s=10, cmap='cividis')
axes[1].set_title('Per-element scaling pattern')
axes[1].set_xlabel('Longitude [deg]')
axes[1].set_ylabel('Latitude [deg]')
fig.colorbar(sc, ax=axes[1], label='scale')
plt.show()


/var/folders/7r/n_x0ntj511v_0gt816mgrc1c0000gq/T/ipykernel_91368/4231509721.py:16: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


## Disk-integrated ``PhysicalLineEmulator`` example
The same spotted mesh can be pushed through ``PhysicalLineEmulator`` to see how the Ca II 8542 Å line reacts to the bright ring.
Because the plage wraps around an off-axis latitude, it deepens the blue wing when that sector rotates toward the observer, producing a subtle asymmetry in the disk-integrated line.


### Wrapping the Ca II emulator

To keep the notebook self-contained we can build a tiny helper that configures a ``PhysicalLineEmulator`` for the Ca II IRT line.
It exposes the Ca-specific knobs (line center, depth, damping, etc.) while forwarding any other keyword arguments to ``PhysicalLineEmulator`` so extending the spectral model stays trivial.


In [7]:
LINE_CENTER = 8542.09

emulator = CaIrtEmulator(
    line_center=LINE_CENTER,
    line_depth=0.45,
    atomic_mass=40.078,
    gamma0=0.07,
    limb_coeffs=(0.3, 0.2),
    use_convective_shift=True,
    v_conv0=1.2,
    v_micro=2.0
)

/Users/mjablons/Documents/stellar-mesh-integration/src/spice/spectrum/physical_line_emulator.py:182: UserWarning: Explicitly requested dtype <class 'jax.numpy.float64'> requested in array is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  self.limb_coeffs = jnp.array(limb_coeffs, dtype=jnp.float64)


In [8]:
from spice.spectrum.spectrum import simulate_observed_flux


log_wavelengths = jnp.linspace(jnp.log10(LINE_CENTER - 1), jnp.log10(LINE_CENTER + 1), 600)
wavelength = np.array(10 ** log_wavelengths)

spectra = simulate_observed_flux(emulator.intensity, spotted_mesh, log_wavelengths)

In [9]:
from spice.models.mesh_transform import add_rotation, evaluate_rotation


m_rotated = add_rotation(spotted_mesh, 10)

# Assuming 10 km/s is the equatorial rotational velocity,
# and the period is the time it takes to make a full rotation (in seconds).
# We also need the radius at which this velocity applies. Let's use the mean radius of the mesh:
radius = 1.0 # in solar radii
radius_m = radius * 6.957e8  # convert solar radii to meters

v_eq = 10e3  # 10 km/s in m/s
period_sec = 2 * np.pi * radius_m / v_eq
print(f"Period for v_eq = 10 km/s: {period_sec:.2f} seconds ({period_sec/86400:.2f} days)")

times = jnp.linspace(0.0, period_sec, 100)

rotated_models = [evaluate_rotation(m_rotated, t) for t in times]

Period for v_eq = 10 km/s: 437121.20 seconds (5.06 days)


In [10]:
rotated_spectra = [simulate_observed_flux(emulator.intensity, s, log_wavelengths) for s in rotated_models]

In [11]:
for s in rotated_spectra:
    plt.plot(wavelength, s[:, 0]/s[:, 1])
plt.show()

/var/folders/7r/n_x0ntj511v_0gt816mgrc1c0000gq/T/ipykernel_91368/833358053.py:3: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [12]:
rotated_spectra = np.array(rotated_spectra)

In [13]:
from spice.plots.plot_mesh import animate_mesh_and_spectra


animate_mesh_and_spectra(rotated_models, rotated_spectra[:, :, 0]/rotated_spectra[:, :, 1],
                         wavelength, property=0, property_label='Temperature [K]', filename='ca_spot.gif',
                         plot_legend=False)

'ca_spot.gif'